# COVID 19 Stats -- Data by Robert Koch Institut, processed by [NPGEO](https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0)

## Boilerplate

In [1]:
import datetime

import pandas as pd
import numpy as np
import qgrid

%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


def prepare(col: str, refcol: str, df: pd.DataFrame) -> pd.Series:
    """Generate a series that can be use with the standar sum() function
    
    The documentation of the dataset specifies that "AnzahlFall" and other
    columns should not be simply summed up.
    """
    # Uses numpy's vectorize() for massive performance improvements.
    # Does the same thing as:
    # return pd.Series((row[col] if row[refcol] in [0,1] else 0) for _, row in df.iterrows())
    
    def preparation(col, refcol):
        return col if refcol == 0 or refcol == 1 else 0
    prep = np.vectorize(preparation)
    return pd.Series(prep(df[col].to_numpy(), df[refcol].to_numpy()))
    

## Load main data source

Data License: Robert Koch-Institut (RKI), [dl-de/by-2-0](https://www.govdata.de/dl-de/by-2-0).
Provided by [NPGEO](https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0).

In [2]:
def get_cases(date: datetime.date, url_prefix="https://raw.githubusercontent.com/woefe/covid19stats/autodata/data/silo") -> pd.DataFrame:
    date_str = date.isoformat()
    rki_cases = pd.read_csv(f"{url_prefix}/{date_str}.csv")
    rki_cases["Datenstand"] = pd.to_datetime(rki_cases["Datenstand"], format="%d.%m.%Y, %H:%M Uhr")
    rki_cases["Meldedatum"] = pd.to_datetime(rki_cases["Meldedatum"], format="%Y/%m/%d %H:%M:%S")
    rki_cases["Refdatum"] = pd.to_datetime(rki_cases["Refdatum"], format="%Y/%m/%d %H:%M:%S")
    rki_cases["cases"] = prepare("AnzahlFall", "NeuerFall", rki_cases)
    rki_cases["deaths"] = prepare("AnzahlTodesfall", "NeuerTodesfall", rki_cases)
    rki_cases["recovered"] = prepare("AnzahlGenesen", "NeuGenesen", rki_cases)
    return rki_cases

rki_cases = get_cases(datetime.date(2020,9,21))

## Load county data
Used to get number of residents and size in km² of every county.
Dataset is taken from [NPGEO](https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/917fc37a709542548cc3be077a786c17_0)

In [3]:
rki_lkr = pd.read_csv("data/RKI_Corona_Landkreise.csv")

## Load state data
The destatis dataset is used to get number of residents and size in km² of every german state.
Dataset is extracted from [destatis](https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/02-bundeslaender.xlsx?__blob=publicationFile).

In [4]:
destatis = pd.read_csv("data/destatis_bundeslaender.csv")

## Overview States

In [5]:
cases = rki_cases.groupby("Bundesland").sum()[["cases", "deaths", "recovered"]]
states = pd.merge(cases, destatis[["Bundesland", "Flaeche", "Einwohner"]], on="Bundesland")
states["cases/km²"] = states["cases"]/states["Flaeche"]
states["cases/inhabitants"] = states["cases"]/states["Einwohner"]
states["deaths/km²"] = states["deaths"]/states["Flaeche"]
states["deaths/inhabitants"] = states["deaths"]/states["Einwohner"]
states.drop(columns=["Flaeche", "Einwohner"], inplace=True)
qgrid.show_grid(states, show_toolbar=True)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Overview Counties (last seven days)

In [6]:
start_at = pd.to_datetime(datetime.date.today() - datetime.timedelta(days=7))
lkr = rki_lkr[["county", "EWZ", "KFL"]]
sum_lkr = rki_cases[rki_cases.Meldedatum > start_at].groupby("Landkreis").sum()[["cases", "deaths", "recovered"]]
counties = pd.merge(lkr, sum_lkr, right_on="Landkreis", left_on="county")
counties["cases/100000 inhabitants"] = (counties["cases"]/counties["EWZ"])*100000
counties["cases/km²"] = counties["cases"]/counties["KFL"]
counties["cases/inhabitants"] = counties["cases"]/counties["EWZ"]
counties["deaths/km²"] = counties["deaths"]/counties["KFL"]
counties["deaths/inhabitants"] = counties["deaths"]/counties["EWZ"]
counties.drop(["EWZ", "KFL"], axis="columns", inplace=True)
qgrid.show_grid(counties, show_toolbar=True)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Cases, recovered, deaths (all of Germany)

In [7]:
#datecol = "Refdatum"
datecol = "Meldedatum"

#per_day = rki_cases[rki_cases["Bundesland"] == "Bayern"].copy()
per_day = rki_cases.copy()

per_day[datecol] = rki_cases[datecol].dt.floor("D")
per_day = per_day.groupby(datecol).sum()[["cases", "deaths", "recovered"]]
per_day.sort_values(datecol, inplace=True)
per_day["cum_cases"] = per_day["cases"].cumsum()
per_day["cum_deaths"] = per_day["deaths"].cumsum()
per_day["cum_recovered"] = per_day["recovered"].cumsum()
per_day.tail(5)

,cases,deaths,recovered,cum_cases,cum_deaths,cum_recovered
Meldedatum,,,,,,
2020-09-16,2161,0,84,267043,9386,242124
2020-09-17,2229,0,43,269272,9386,242167
2020-09-18,1870,0,31,271142,9386,242198
2020-09-19,996,0,12,272138,9386,242210
2020-09-20,199,0,0,272337,9386,242210


In [8]:
fig, ax = plt.subplots(figsize=(10,8))
import matplotlib.dates as mdates

plt.plot(per_day.index, per_day.cum_cases, label="Cases")
plt.plot(per_day.index, per_day.cum_deaths, label="Deaths")

# The recovered cases should "lag behind" by approx., but it doesn't.
# That's because the dataset does not have a date indicating when a patient has recovered.
# Instead the original record seems to be updated.
# Hence, the I won't plot the recovered cases.
#plt.plot(per_day.index, per_day.cum_recovered, label="Recovered")
ax.get_xaxis().set_major_locator(mdates.DayLocator(interval=5))
ax.get_xaxis().set_minor_locator(mdates.DayLocator(interval=1))

fig.autofmt_xdate()
plt.legend()
plt.title("Cumulative cases (all of Germany)")
plt.grid()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fig, ax = plt.subplots(figsize=(10,8))

plt.bar(per_day.index, per_day.cases, label="Cases")
#plt.plot(per_day.index, per_day.cum_deaths, label="Deaths")
#plt.plot(per_day.index, per_day.cum_recovered, label="Recovered")
ax.get_xaxis().set_major_locator(mdates.DayLocator(interval=5))
ax.get_xaxis().set_minor_locator(mdates.DayLocator(interval=1))

fig.autofmt_xdate()
plt.legend()
plt.title("Registered cases per day (all of Germany)")
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Registerd cases per state

In [10]:
states = list(rki_cases.drop_duplicates("Bundesland").Bundesland)

#datecol = "Refdatum"
datecol = "Meldedatum"

def plot_data(state):
    df = rki_cases[rki_cases["Bundesland"] == state].copy()

    df[datecol] = rki_cases[datecol].dt.floor("D")
    df = df.groupby(datecol).sum()
    df.sort_values(datecol, inplace=True)
    return df

fig, ax = plt.subplots(figsize=(10,8))

for state in states:
    df = plot_data(state)
    plt.plot(df.index, df.cases, label=state)

# The recovered cases should "lag behind" by approx., but it doesn't.
# That's because the dataset does not have a date indicating when a patient has recovered.
# Instead the original record seems to be updated.
# Hence, the I won't plot the recovered cases.
#plt.plot(per_day.index, per_day.cum_recovered, label="Recovered")
ax.get_xaxis().set_major_locator(mdates.DayLocator(interval=5))
ax.get_xaxis().set_minor_locator(mdates.DayLocator(interval=1))

fig.autofmt_xdate()
plt.legend()
plt.title("Registered cases")
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Grouped by age (all of Germany, entire dataset)

In [11]:
by_age = rki_cases.groupby("Altersgruppe").sum()[["cases", "deaths", "recovered"]]
by_age["sick"] = by_age.cases - by_age.deaths - by_age.recovered
by_age["deathrate %"] = (by_age.deaths / by_age.cases) * 100
by_age

,cases,deaths,recovered,sick,deathrate %
Altersgruppe,,,,,
A00-A04,4822,1,4304,517,0.020738
A05-A14,12690,0,11107,1583,0.000000
A15-A34,83803,19,74869,8915,0.022672
A35-A59,107054,448,99505,7101,0.418480
A60-A79,41010,3034,36024,1952,7.398196
A80+,22767,5884,16283,600,25.844424
unbekannt,191,0,118,73,0.000000


In [12]:
# Overview over entire data. Sanity check. Should be the same data as on the RKI dashboard
overall = by_age[["cases", "deaths", "recovered", "sick"]].sum(axis="rows")
print("overall")
print(f"   - cases:     {overall.cases}")
print(f"   - deaths:    {overall.deaths}")
print(f"   - recovered: {overall.recovered}")
print(f"   - sick:      {overall.sick}")
print(f"   - deathrate: {overall.deaths / overall.cases}")

overall
   - cases:     272337
   - deaths:    9386
   - recovered: 242210
   - sick:      20741
   - deathrate: 0.03446465225070409


## Cases per week in a county

In [13]:
county = "SK München"
county_data = rki_cases[rki_cases.Landkreis == county].copy()
county_data["week"] = county_data["Meldedatum"].apply(lambda d: d.isocalendar()[1])
county_data.groupby("week").sum()[["cases", "deaths", "recovered"]]

,cases,deaths,recovered
week,,,
5,1,0,1
6,1,0,1
7,1,0,1
9,2,0,2
10,22,0,22
11,224,1,223
12,1055,12,1043
13,1336,34,1302
14,1193,54,1139


## Estimated active cases over time

In [14]:
import calendar
import os
from multiprocessing import Pool
from typing import Tuple

cal = calendar.Calendar()

def get_active_cases(date: datetime.date, url_prefix="/tmp/tmp.71robUWnfn/silo") -> Tuple[str,int]:
    date_str = date.isoformat()
    try:
        rki_cases = pd.read_csv(f"{url_prefix}/{date_str}.csv")
    except:
        return date_str, -1
    cases = prepare("AnzahlFall", "NeuerFall", rki_cases).sum()
    deaths = prepare("AnzahlTodesfall", "NeuerTodesfall", rki_cases).sum()
    recovered = prepare("AnzahlGenesen", "NeuGenesen", rki_cases).sum()
    return date_str, cases - deaths - recovered

dates = (date 
    for month in range(3,10)
    for date in cal.itermonthdates(2020, month)
)

with Pool(len(os.sched_getaffinity(0))) as pool:
    cases_per_day = pool.map(get_active_cases, dates)

In [15]:
active_daily = pd.DataFrame(cases_per_day, columns=["date", "active_cases"]).sort_values("date")
fig, ax = plt.subplots(figsize=(10,8))

plt.plot(active_daily.date, active_daily.active_cases, label="Estimated Active Cases")

ax.get_xaxis().set_major_locator(mdates.DayLocator(interval=5))
ax.get_xaxis().set_minor_locator(mdates.DayLocator(interval=1))

fig.autofmt_xdate()
plt.legend()
plt.title("Estimated Active Cases every day (all of Germany)")
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …